# Virtual Screening

In [1]:
from pymol import cmd
import py3Dmol

from vina import Vina

import pandas as pd
import numpy as np

from openbabel import pybel

from rdkit import Chem
from rdkit.Chem import AllChem,rdFMCS, Draw

from meeko import MoleculePreparation
from meeko import obutils

import MDAnalysis as mda
from MDAnalysis.coordinates import PDB

import prolif as plf
from prolif.plotting.network import LigNetwork


import sys, os, random
sys.path.insert(1, 'utilities/')

from multiprocessing import Pool

from utils import fix_protein, getbox, generate_ledock_file, dok_to_sdf

import warnings
warnings.filterwarnings("ignore")

%config Completer.use_jedi = False

In [2]:
os.chdir('test/Virtual_Screening/')

In [3]:
cmd.fetch(code='1X1R',type='pdb1')
cmd.select(name='Prot',selection='polymer.protein')
cmd.select(name='GDP',selection='organic')
cmd.save(filename='1X1R_clean.pdb',format='pdb',selection='Prot')
cmd.save(filename='1X1R_GDP.mol2',format='mol2',selection='GDP')
cmd.delete('all')

 PyMOL not running, entering library mode (experimental)


In [4]:
fix_protein(filename='1X1R_clean.pdb',addHs_pH=7.4,try_renumberResidues=True,output='1X1R_clean_H.pdb')

In [5]:
view = py3Dmol.view()
view.removeAllModels()
view.setViewStyle({'style':'outline','color':'black','width':0.1})

view.addModel(open('1X1R_clean_H.pdb','r').read(),format='pdb')
Prot=view.getModel()
Prot.setStyle({'cartoon':{'arrows':True, 'tubes':True, 'style':'oval', 'color':'white'}})
view.addSurface(py3Dmol.VDW,{'opacity':0.6,'color':'white'})

view.addModel(open('1X1R_GDP.mol2','r').read(),format='mol2')
ref_m = view.getModel()
ref_m.setStyle({},{'stick':{'colorscheme':'greenCarbon','radius':0.2}})

view.zoomTo()
view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [6]:
!../../bin/prepare_receptor -v -r 1X1R_clean_H.pdb -A hydrogens -o 1X1R_clean_H.pdbqt

set verbose to  True
set receptor_filename to  1X1R_clean_H.pdb
set repairs to  hydrogens
set outputfilename to  1X1R_clean_H.pdbqt
read  1X1R_clean_H.pdb
setting up RPO with mode= automatic and outputfilename=  1X1R_clean_H.pdbqt
charges_to_add= gasteiger
delete_single_nonstd_residues= None
adding gasteiger charges to peptide


In [7]:
smiles=['C1=NC(=C2C(=N1)N(C=N2)CCOCP(=O)(O)O)N',
        'C1=NC(=C2C(=N1)N(C=N2)[C@H]3[C@@H]([C@@H]([C@H](O3)COP(=O)(O)O)O)O)N',
        'C[C@@H](C1=CC2=C(C(=CC=C2)Cl)C(=O)N1C3=CC=CC=C3)NC4=NC=NC5=C4NC=N5',
        'C1=NC(=C2C(=N1)N(C=N2)C3C(C(C(O3)CO)O)O)N',
        'C[C@H](CN1C=NC2=C(N=CN=C21)N)OC[P@@](=O)(N[C@@H]',
        'C1=NC(=C2C(=N1)N(C=N2)[C@H]3[C@@H]([C@@H]([C@H](O3)C[C@H](CC[C@@H](C(=O)O)N)N)O)O)N',
        'C[S+](CC[C@@H](C(=O)O)N)C[C@@H]1[C@H]([C@H]([C@@H](O1)N2C=NC3=C(N=CN=C32)N)O)O',
        'CN1C=C(C(=N1)OC)NC2=C3C(=NC(=N2)N4C[C@H]([C@@H](C4)F)NC(=O)C=C)N(C=N3)C',
        'C1COC[C@@H]1NC2=C3C(=NC=N2)N(C=N3)[C@H]4[C@@H]([C@@H]([C@H](O4)CO)O)O']

In [8]:
for index,smi in enumerate(smiles):
    mol=pybel.readstring(string=smi,format='smiles')
    mol.make3D()
    out=pybel.Outputfile(filename='mol_'+str(index)+'.pdbqt',format='pdbqt',overwrite=True)
    out.write(mol)
    out.close()

In [9]:
cmd.load(filename='1X1R_clean_H.pdb',format='pdb',object='prot')
cmd.load(filename='1X1R_GDP.mol2',format='mol2',object='lig')

center,size=getbox(selection='lig',extending=6.0,software='vina')
cmd.delete('all')
print(center)
print(size)

{'center_x': 3.4204999953508377, 'center_y': 9.91599988937378, 'center_z': 11.27299976348877}
{'size_x': 19.56700000166893, 'size_y': 18.30399990081787, 'size_z': 23.20599937438965}


In [10]:
def vina(receptor='',ligand='',center=[0,0,0],size=[0,0,0],exhaustiveness=8,n_poses=10,output=''):
    v = Vina(sf_name='vina')

    v.set_receptor(receptor)

    v.set_ligand_from_file(ligand)

    v.compute_vina_maps(center=center, box_size=size)

    v.dock(exhaustiveness=exhaustiveness, n_poses=n_poses)
    v.write_poses(output, n_poses=n_poses, overwrite=True)

In [11]:
for file in os.listdir('./'):
    if 'mol_' in file:
        vina(receptor='1X1R_clean_H.pdbqt',
             ligand=file,
             center=[center['center_x'],center['center_y'],center['center_z']],
             size=[size['size_x'],size['size_y'],size['size_z']],
             exhaustiveness=8,
             n_poses=10,
             output='vina_outfiles/'+file.replace('.pdbqt','_vina_out.pdbqt'))

In [12]:
for file in os.listdir('vina_outfiles/'):
    if 'vina' in file:
        results = [m for m in pybel.readfile(filename='vina_outfiles/'+file,format='pdbqt')]
        out=pybel.Outputfile(filename='vina_outfiles/'+file.replace('pdbqt','sdf'),format='sdf',overwrite=True)
        for pose in results:
            out.write(pose)
        out.close()

In [13]:
view = py3Dmol.view()
view.removeAllModels()
view.setViewStyle({'style':'outline','color':'black','width':0.1})

view.addModel(open('1X1R_clean_H.pdb','r').read(),'pdb')
Prot=view.getModel()
Prot.setStyle({'cartoon':{'arrows':True, 'tubes':True, 'style':'oval', 'color':'white'}})
view.addSurface(py3Dmol.VDW,{'opacity':0.8,'color':'white'})


view.addModel(open('1X1R_GDP.mol2','r').read(),'mol2')
ref_m = view.getModel()
ref_m.setStyle({},{'stick':{'colorscheme':'greenCarbon','radius':0.2}})

for file in os.listdir('vina_outfiles/'):
    if 'sdf' in file:
        pose_1=Chem.SDMolSupplier('vina_outfiles/'+file,False)[0]
        p=Chem.MolToMolBlock(pose_1)
        color = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])]
        view.addModel(p,'mol')
        z= view.getModel()
        z.setStyle({},{'stick':{'color':color[0],'radius':0.05,'opacity':0.6}})

view.zoomTo()
view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [14]:
smiles=['C1=NC(=C2C(=N1)N(C=N2)CCOCP(=O)(O)O)N',
        'C1=NC(=C2C(=N1)N(C=N2)[C@H]3[C@@H]([C@@H]([C@H](O3)COP(=O)(O)O)O)O)N',
        'C[C@@H](C1=CC2=C(C(=CC=C2)Cl)C(=O)N1C3=CC=CC=C3)NC4=NC=NC5=C4NC=N5',
        'C1=NC(=C2C(=N1)N(C=N2)C3C(C(C(O3)CO)O)O)N',
        'C[C@H](CN1C=NC2=C(N=CN=C21)N)OC[P@@](=O)(N[C@@H]',
        'C1=NC(=C2C(=N1)N(C=N2)[C@H]3[C@@H]([C@@H]([C@H](O3)C[C@H](CC[C@@H](C(=O)O)N)N)O)O)N',
        'C[S+](CC[C@@H](C(=O)O)N)C[C@@H]1[C@H]([C@H]([C@@H](O1)N2C=NC3=C(N=CN=C32)N)O)O',
        'CN1C=C(C(=N1)OC)NC2=C3C(=NC(=N2)N4C[C@H]([C@@H](C4)F)NC(=O)C=C)N(C=N3)C',
        'C1COC[C@@H]1NC2=C3C(=NC=N2)N(C=N3)[C@H]4[C@@H]([C@@H]([C@H](O4)CO)O)O']

In [15]:
for index,smi in enumerate(smiles):
    mol=pybel.readstring(string=smi,format='smiles')
    mol.make3D()
    mol.addh()
    out=pybel.Outputfile(filename='ledock_inputfiles/'+'mol_'+str(index)+'.mol2',format='mol2',overwrite=True)
    out.write(mol)
    out.close()

In [16]:
cmd.load(filename='1X1R_clean_H.pdb',format='pdb',object='prot')
cmd.load(filename='1X1R_GDP.mol2',format='mol2',object='lig')

X,Y,Z=getbox(selection='lig',extending=6.0,software='ledock')
cmd.delete('all')
print(X)
print(Y)
print(Z)

{'minX': -6.363000005483627, 'maxX': 13.203999996185303}
{'minY': 0.7639999389648438, 'maxY': 19.067999839782715}
{'minZ': -0.3299999237060547, 'maxZ': 22.875999450683594}


In [17]:
l_list=[]
for file in os.listdir('ledock_inputfiles/'):
    if 'mol2' in file:
        l_list.append('ledock_inputfiles/'+file+'\n')

In [18]:
l_list

['ledock_inputfiles/mol_0.mol2\n',
 'ledock_inputfiles/mol_1.mol2\n',
 'ledock_inputfiles/mol_2.mol2\n',
 'ledock_inputfiles/mol_3.mol2\n',
 'ledock_inputfiles/mol_4.mol2\n',
 'ledock_inputfiles/mol_5.mol2\n',
 'ledock_inputfiles/mol_6.mol2\n',
 'ledock_inputfiles/mol_7.mol2\n',
 'ledock_inputfiles/mol_8.mol2\n']

In [19]:
generate_ledock_file(receptor='1X1R_clean_H.pdb',l_list=l_list,
                     l_list_outfile='ligand.list',
                     x=[X['minX'],X['maxX']],
                     y=[Y['minY'],Y['maxY']],
                     z=[Z['minZ'],Z['maxZ']],
                     n_poses=10,
                     rmsd=1.0,
                     out='dock.in')

In [20]:
!../../bin/ledock_linux_x86 dock.in

In [21]:
for file in os.listdir('ledock_inputfiles/'):
    if 'dok' in file:
        os.rename('ledock_inputfiles/'+file,'ledock_outfiles/'+file)

In [22]:
for file in os.listdir('ledock_outfiles/'):
    if 'dok' in file:
        dok_to_sdf(dok_file='ledock_outfiles/'+file,output='ledock_outfiles/'+file.replace('dok','sdf'))

In [23]:
view = py3Dmol.view()
view.removeAllModels()
view.setViewStyle({'style':'outline','color':'black','width':0.1})

view.addModel(open('1X1R_clean_H.pdb','r').read(),'pdb')
Prot=view.getModel()
Prot.setStyle({'cartoon':{'arrows':True, 'tubes':True, 'style':'oval', 'color':'white'}})
view.addSurface(py3Dmol.VDW,{'opacity':0.8,'color':'white'})


view.addModel(open('1X1R_GDP.mol2','r').read(),'mol2')
ref_m = view.getModel()
ref_m.setStyle({},{'stick':{'colorscheme':'greenCarbon','radius':0.2}})

for file in os.listdir('ledock_outfiles/'):
    if 'sdf' in file:
        pose_1=Chem.SDMolSupplier('ledock_outfiles/'+file,False)[0]
        p=Chem.MolToMolBlock(pose_1)
        color = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])]
        view.addModel(p,'mol')
        z= view.getModel()
        z.setStyle({},{'stick':{'color':color[0],'radius':0.05,'opacity':0.6}})

view.zoomTo()
view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol